## Part IV Decompounded Options

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import interpolate
from math import log, sqrt, exp
from scipy import integrate

### Read dataframe

In [2]:
data_FS = pd.read_csv('df_ForwardSwap.csv', header = 0, index_col = 0)  
data_DF = pd.read_csv('df_comb.csv', header = 0, index_col = 0)  
data_SABR = pd.read_csv('df_SABRImpVol.csv', header = 0, index_col = 0)
data_SABR = data_SABR.rename(columns={'α': 'Alpha','ρ':'Rho','ν':'Nu'})

### define models

In [3]:
def Black76Lognormal(F, K, r, sigma, T, opt):
    d1 = (log(F/K)+(sigma*sigma/2)*T)/(sigma*sqrt(T))
    d2 = d1-sigma*sqrt(T)
    if opt == 'Call':
        return F*exp(-r*T)*norm.cdf(d1) - K*exp(-r*T)*norm.cdf(d2)
    elif opt == 'Put':
        return K*exp(-r*T)*norm.cdf(-d2) - F*exp(-r*T)*norm.cdf(-d1)

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*log(F/X)
        zhi = log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

### define IRRs

$IRR  (K) = \sum_{i = 1}^{N * m} \frac{1}{m} *({1 + \frac{K}{m}})^{-i}\ \ \ \ \    (delta = \frac{1}{m})$

$IRR' (K) = \sum_{i = 1}^{N * m} (-i)*(\frac{1}{m})^2 *({1 + \frac{K}{m}})^{-i-1}\ \ \ \ \    (delta = \frac{1}{m})$

$IRR'' (K) = \sum_{i = 1}^{N * m} (-i)*(-i-1) *(\frac{1}{m})^3 *({1 + \frac{K}{m}})^{-i-2}\ \ \ \ \    (delta = \frac{1}{m})$

In [4]:
def IRR(K, Tenor, Delta):
    total = 0
    for i in np.arange(1, Tenor/Delta +1):
        total += Delta*(1 + K* Delta)**(-i)
    return total

def IRRf(K, Tenor, Delta):  
    total = 0
    for i in np.arange(1, Tenor/Delta +1):
        total += (-i)*(Delta**2)*(1 + K* Delta)**(-i-1)
    return total

def IRRff(K, Tenor, Delta):
    total = 0
    for i in np.arange(1, Tenor/Delta +1):
        total += (-i)*(-i-1)*(Delta**3)*(1 + K* Delta)**(-i-2)
    return total

### Question 1   

#### Use static replication to value the PV of payoff: $CMS\ 10y^{1/4} - 0.04^{1/2}$ at time T = 5y

$g(F) = F^{\frac{1}{4}} - 0.2 \ \ \ \ \ \ g'(F) = \frac{1}{4}F^{-\frac{3}{4}} \ \ \ \ \ \ \ g''(F) = -\frac{3}{16}F^{-\frac{7}{4}}$

$ h(K) = \frac{g(K)}{IRR(K)}$ <br><br>
$ h'(K) = \frac{IRR(K)g'(K) - g(K)IRR'(K)}{IRR(K)^2}$<br><br>
$ h''(K) = \frac{IRR(K)g''(K) - IRR''(K)g(K) - 2 * IRR'(K)g'(K)}{IRR(K)^2} +\frac{2*IRR'(K)^2g(K)}{IRR(K)^3}$

In [5]:
#define payoff functions g(x) and h(x)
def g(K):
    return K**(1/4) - 0.2

def gf(K):
    return (1/4)*(K)**(-3/4)

def gff(K):
    return (-3/16)*(K)**(-7/4)

def h(K,Tenor,Delta):
    return g(K)/IRR(K, Tenor, Delta)

def hf(K, Tenor, Delta):
    top = IRR(K, Tenor, Delta)*gf(K)-g(K)*IRRf(K, Tenor, Delta)
    bottom = IRR(K, Tenor, Delta)**2
    return top / bottom

def hff(K, Tenor, Delta):
    top = IRR(K,Tenor,Delta)*gff(K)-IRRff(K,Tenor,Delta)*g(K)-2*IRRf(K,Tenor,Delta)*gf(K)
    bottom = IRR(K,Tenor,Delta)**2
    term3 = 2*IRRf(K,Tenor,Delta)**2*g(K)/IRR(K,Tenor,Delta)**3
    return top/bottom + term3

$ V_0  = D(0,T)g(F) + \int_0^F h''(K)V^{rec}(K)dK + \int_F^\infty h''(K)V^{pay}(K)dK $

In [6]:
F = data_FS.loc[9, 'Swap_Rate']   # this is 5y x 10y forward swap rate
D = data_DF.loc[9, 'OIS_DF']      # this is OIS discount factor D(0, 5y)

alpha = data_SABR.loc[9, 'Alpha'] # this is alpha parameter for 5y x 10y forward swap
beta = 0.9                        # this is pre-determined beta at 0.9
rho = data_SABR.loc[9, 'Rho']     # this is rho parameter for 5y x 10y forward swap
nu = data_SABR.loc[9,  'Nu']      # this is nu parameter for 5y x 10y forward swap

Tenor = 10
Delta = 0.5
T = 5

V_rec = integrate.quad(lambda x: hff(x,Tenor,Delta)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T, "Put"),
                                                    0,F)
V_pay = integrate.quad(lambda x: hff(x,Tenor,Delta)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T, "Call"),
                                                    F,1000)
                                                   
PVoption = D * g(F) + V_rec[0] + V_pay[0]

In [7]:
print("PV is", PVoption)

PV is 0.24975332100232644


### Question 2

#### Use static replication to value the PV of this payoff: $(CMS\ 10y^{1/4} - 0.04^{1/2})^+$

$F^\frac{1}{4} > 0.2$<br>
$F > 0.2^4$<br>
$F > 0.0016 = L$

$CMS\ Caplet = h'(L)V^{pay}(L) + \int^{\infty}_Lh''(K)V^{pay}(K)dK$

In [8]:
Tenor = 10
Delta = 0.5
T = 5
L = 0.2**4

term1 = hf(L, Tenor, Delta)*Black76Lognormal(F, L, 0, SABR(F, L, T, alpha, 0.9, rho, nu),T,"Call")
term2 = integrate.quad(lambda x: hff(x, Tenor,Delta)*Black76Lognormal(F, x, 0, SABR(F, x, T, alpha, 0.9, rho, nu),T,"Call"),
                                                    L,1000)
PV_caplet = term1+ term2[0]
PV_caplet

0.031198138039831624